In [6]:
from torch import nn
from mads_datasets import DatasetFactoryProvider, DatasetType
import torch.optim as optim
from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics
from tomlserializer import TOMLSerializer


In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-09-22 20:38:24.097 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist
2025-09-22 20:38:24.100 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist\fashionmnist.pt


In [3]:
accuracy = metrics.Accuracy()

In [4]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps= 100,
    valid_steps= 100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

In [9]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

In [10]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2025-09-22 20:46:07.106 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250922-204607
2025-09-22 20:46:09.215 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:01<00:00, 88.94it/s]
2025-09-22 20:46:10.921 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9631 test 0.6335 metric ['0.7683']
100%|██████████| 100/100 [00:01<00:00, 68.73it/s]
2025-09-22 20:46:12.794 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5732 test 0.5569 metric ['0.8067']
100%|██████████| 100/100 [00:01<00:00, 77.49it/s]
2025-09-22 20:46:14.458 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5218 test 0.5131 metric ['0.8122']
100%|██████████| 3/3 [00:05<00:00,  1.67s/it]
